In [ ]:
import csv
import os
from os import path
import re
import pytesseract
from pytesseract import Output
import cv2
import imutils
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\tcheung\AppData\Local\Tesseract-OCR\tesseract.exe'
# custom_oem_psm_config = r'--oem 3'
custom_oem_psm_config = r'tessedit_char_whitelist=12345678ABCDEFGHIJKLMNOPQRSTUVWXYZ load_system_dawg=false load_freq_dawg=false'

In [ ]:
with open("attachments_amount_jwt_output.csv") as f:
    reader = csv.DictReader(f)
    receipt_list = list(reader)

In [ ]:
def img_resize(inputImage):
    (H, W) = inputImage.shape[:2]
    orig_rH = H/W
    default = 4320
    
    if H >= W:
        (newW, newH) = (int(round(default * W/H)), default)
    
    if H < W:
        (newW, newH) = (default, int(round(default * H/W)))

    # resize the image and grab the new image dimensions
    img = cv2.resize(inputImage, (newW, newH))
    return img

In [ ]:
def img_resize_for_show(inputImage):
    (H, W) = inputImage.shape[:2]
    orig_rH = H/W
    default = 1080
    
    if H >= W:
        (newW, newH) = (int(round(default * W/H)), default)
    
    if H < W:
        (newW, newH) = (default, int(round(default * H/W)))

    # resize the image and grab the new image dimensions
    img = cv2.resize(inputImage, (newW, newH))
    return img

In [ ]:
price_char = set(['$', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.', ','])

def has_number(inputString):
    return any(char.isdigit() for char in inputString)

def crop_number_from_str(text):
    num_id = [i for i, char in enumerate(text) if char.isdigit()]
    start, end = min(num_id), max(num_id) + 1
    if start > 0 and  text[start - 1] == "-":
        start -= 1
    return start, end

def is_price(text):
    if any([char not in price_char for char in text]):
        return False
    if not has_number(text):
        return False
    # decimal or comma must be in price for it to be interpreted as price
    if not any([char in ['.',','] for char in text]):
        return False
    if len(text.replace(',', '.').rsplit('.')[-1]) < 2:
        return False
#     start, end = crop_number_from_str(text)
#     if end - start > 6:
#         return False
#     try:
#         num = float(text[start: end])
#     except ValueError:
#         return False
#     if num > 2000:
#         return False
    return True

In [ ]:
img_list = list()

for i in range(len(receipt_list)):
    if receipt_list[i]['filetype'] in ['jpeg','png']:
        # load the input image and grab the image dimensions
        if receipt_list[i]['filetype'] in ['png']:
            img = cv2.imread(os.path.join(
                'images_unshadow', 
                receipt_list[i]['filename'] + '.' + receipt_list[i]['filetype']))
        
        if receipt_list[i]['filetype'] in ['jpeg']:
            img = cv2.imread(os.path.join(
                'images_unshadow', 
                receipt_list[i]['filename'] + '.' + 'jpg'))
        
        # extract text data from image
        d = pytesseract.image_to_data(img, output_type=Output.DICT, config=custom_oem_psm_config)
        
        # find max currency/price figure in text data
        iterations = len(d['text'])
        temp_price_list = list()
                
        for k in range(iterations):
            if is_price(str(d['text'][k])):
                step1 = d['text'][k].replace('$','').replace(",", ".")
                step2 = step1.replace(".", "", 
                              max(step1.count(".") -1, 0))
                temp_price_list.append(float(step2))

        max_price = max(temp_price_list, default=0)
        
        # input results in list of dictionaries
        img_dict = dict()
        
        img_dict['i'] = i
        img_dict['total_pred'] = max_price
        img_dict['amount'] = (receipt_list[i]['amount'])
        img_dict['filename'] = (receipt_list[i]['filename'])
        img_dict['filetype'] = (receipt_list[i]['filetype'])
        
        img_list.append(img_dict)
        


In [ ]:
img_list

In [ ]:
i = 13

if receipt_list[i]['filetype'] in ['png']:
    img = cv2.imread(os.path.join(
        'images_unshadow', 
        receipt_list[i]['filename'] + '.' + receipt_list[i]['filetype']))

if receipt_list[i]['filetype'] in ['jpeg']:
    img = cv2.imread(os.path.join(
        'images_unshadow', 
        receipt_list[i]['filename'] + '.' + 'jpg'))

cv2.imshow('img', img)
cv2.waitKey(0)

In [ ]:
receipt_list[i]['filename']

In [ ]:
d = pytesseract.image_to_data(img, output_type=Output.DATAFRAME)

In [ ]:
n_boxes = len(d['level'])

for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imshow('img', img)
cv2.waitKey(0)

In [ ]:
d.to_csv('img_test_result.csv')
d

In [ ]:
# find max currency/price figure in text data
iterations = len(d['text'])
temp_price_list = list()

for k in range(iterations):
    if is_price(str(d['text'][k])):
        step1 = d['text'][k].replace('$','').replace(",", ".")
        step2 = step1.replace(".", "", 
                      max(step1.count(".") -1, 0))
        temp_price_list.append(float(step2) )

temp_price_list

In [ ]:
max_price = max(temp_price_list, default=0)
max_price

In [ ]:
i = 123

if receipt_list[i]['filetype'] in ['png']:
    img = cv2.imread(os.path.join(
        'images_unshadow', 
        receipt_list[i]['filename'] + '.' + receipt_list[i]['filetype']))

if receipt_list[i]['filetype'] in ['jpeg']:
    img = cv2.imread(os.path.join(
        'images_unshadow', 
        receipt_list[i]['filename'] + '.' + 'jpg'))

# img_resize_for_show = img_resize_for_show(img)
cv2.imshow('img', img)
cv2.waitKey(0)

In [ ]:
receipt_list[i]['filename']

In [ ]:
d = pytesseract.image_to_data(img, output_type=Output.DATAFRAME, config=custom_oem_psm_config)

In [ ]:
d.to_csv('img_test_result.csv')
d

In [ ]:
n_boxes = len(d['level'])

for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

img_resize = img_resize_for_show(img)
cv2.imshow('img_resize', img_resize)
cv2.waitKey(0)

In [ ]:
# find max currency/price figure in text data
iterations = len(d['text'])
temp_price_list = list()

for k in range(iterations):
    if is_price(str(d['text'][k])):
        step1 = d['text'][k].replace('$','').replace(",", ".")
        step2 = step1.replace(".", "", 
                      max(step1.count(".") -1, 0))
        temp_price_list.append(float(step2) )

temp_price_list

In [ ]:
max_price = max(temp_price_list, default=0)
max_price

In [ ]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    
    # the top-left point will have the smallest sum, whereas the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # now, compute the difference between the points, the top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

In [ ]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points in the top-left, top-right, bottom-right, and bottom-left order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [ ]:
from pyimagesearch.transform import four_point_transform

In [ ]:
# convert the image to grayscale, blur it, and find edges in the image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 25, 100)
edged1 = cv2.Canny(blurred, 50, 100)
edged2 = cv2.Canny(blurred, 45, 100)
edged3 = cv2.Canny(blurred, 40, 100)
edged4 = cv2.Canny(blurred, 35, 100)
edged5 = cv2.Canny(blurred, 30, 100)

In [ ]:
cv2.imshow('edged1',edged1)
cv2.imshow('edged2',edged2)
cv2.imshow('edged3',edged3)
cv2.imshow('edged4',edged4)
cv2.imshow('edged5',edged5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# find the contours in the edged image, keeping only the largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    # if our approximated contour has four points, then we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
csv_columns = ['i','total_pred','amount','filename','filetype']
with open('results_img_list.csv', mode='w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=csv_columns)
    writer.writeheader()
    for data in img_list:
        writer.writerow(data)